比赛进球数分析

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
matches = spark.read.format("csv").option("header",True).option("encoding","utf-8").option("sep",",").load("../dataset/WorldCupMatches.csv")
matches.head()

Row(Year='1930.0', Datetime='13 Jul 1930 - 15:00 ', Stage='Group 1', Stadium='Pocitos', City='Montevideo ', Home Team Name='France', Home Team Goals='4.0', Away Team Goals='1.0', Away Team Name='Mexico', Attendance='4444.0', Half-time Home Goals='3.0', Half-time Away Goals='0.0', Referee='LOMBARDI Domingo (URU)', Assistant 1='CRISTOPHE Henry (BEL)', Assistant 2='REGO Gilberto (BRA)', RoundID='201.0', MatchID='1096.0', Home Team Initials='FRA', Away Team Initials='MEX')

In [2]:
matches.createTempView("goalsGame")

In [3]:
matches.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Datetime: string (nullable = true)
 |-- Stage: string (nullable = true)
 |-- Stadium: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Home Team Name: string (nullable = true)
 |-- Home Team Goals: string (nullable = true)
 |-- Away Team Goals: string (nullable = true)
 |-- Away Team Name: string (nullable = true)
 |-- Attendance: string (nullable = true)
 |-- Half-time Home Goals: string (nullable = true)
 |-- Half-time Away Goals: string (nullable = true)
 |-- Referee: string (nullable = true)
 |-- Assistant 1: string (nullable = true)
 |-- Assistant 2: string (nullable = true)
 |-- RoundID: string (nullable = true)
 |-- MatchID: string (nullable = true)
 |-- Home Team Initials: string (nullable = true)
 |-- Away Team Initials: string (nullable = true)


In [4]:
top10_goals_match = spark.sql('''
SELECT * FROM goalsGame ORDER BY CAST(`Home Team Goals` AS INTEGER)+CAST(`Away Team Goals` AS INTEGER) DESC LIMIT 10
''')

In [5]:
top10_goals_match.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/worldcup") \
    .option("dbtable", "top10_goals_match") \
    .option("user", "root") \
    .option("password", "") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .mode("append") \
    .save()
top10_goals_match.show()

+------+--------------------+--------------------+------------------+--------------+--------------+---------------+---------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|  Year|            Datetime|               Stage|           Stadium|          City|Home Team Name|Home Team Goals|Away Team Goals|Away Team Name|Attendance|Half-time Home Goals|Half-time Away Goals|             Referee|         Assistant 1|         Assistant 2|RoundID|MatchID|Home Team Initials|Away Team Initials|
+------+--------------------+--------------------+------------------+--------------+--------------+---------------+---------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|1954.0|26 Jun 1954 - 17:00 |      Quarter-finals

In [6]:
top10_biggest_difference_match = spark.sql('''
SELECT * FROM goalsGame ORDER BY ABS(CAST(`Home Team Goals` AS INTEGER)-CAST(`Away Team Goals` AS INTEGER)) DESC LIMIT 10
''')

In [7]:
top10_biggest_difference_match.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/worldcup") \
    .option("dbtable", "top10_biggest_difference_match") \
    .option("user", "root") \
    .option("password", "") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .mode("append") \
    .save()
top10_biggest_difference_match.show()

+------+--------------------+--------------+-----------------+---------------+--------------+---------------+---------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-----------+------------------+------------------+
|  Year|            Datetime|         Stage|          Stadium|           City|Home Team Name|Home Team Goals|Away Team Goals|Away Team Name|Attendance|Half-time Home Goals|Half-time Away Goals|             Referee|         Assistant 1|         Assistant 2|   RoundID|    MatchID|Home Team Initials|Away Team Initials|
+------+--------------------+--------------+-----------------+---------------+--------------+---------------+---------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-----------+------------------+------------------+
|1954.0|17 Jun 1954 - 18:00 |       Group 2|  